#### 라이브러리

In [66]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
import time
from sklearn.metrics import classification_report, confusion_matrix
import warnings
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [67]:
# 경고 메시지 무시
warnings.filterwarnings(action='ignore', category=UserWarning, module='xgboost')

In [68]:
data = pd.read_csv('모델링DB용.csv')
data.head(5)

,curColor,curPer,curColor2,curPer2,curColor3,curPer3,curObject,curHead,curTitle,curExplain,curArtist,song,genre
0,Black,50.38,Grey,22.21,Grey,9.42,바지 시력 관리 고글 색안경 검정색과 흰색 플래시 사진 스타일 안경 단색화 흑백 사진,드라이브 맛 트렌디 세련된 알앤비 국내 힙합,Like I Do,NaN,박재범,박재범 - Like I Do,R&B
1,Black,50.38,Grey,22.21,Grey,9.42,바지 시력 관리 고글 색안경 검정색과 흰색 플래시 사진 스타일 안경 단색화 흑백 사진,드라이브 맛 트렌디 세련된 알앤비 국내 힙합,Rewind,NaN,VVON,VVON - Rewind,R&B
2,Black,50.38,Grey,22.21,Grey,9.42,바지 시력 관리 고글 색안경 검정색과 흰색 플래시 사진 스타일 안경 단색화 흑백 사진,드라이브 맛 트렌디 세련된 알앤비 국내 힙합,Westcoast Marriage,NaN,Austn,Austn - Westcoast Marriage,R&B
3,Black,50.38,Grey,22.21,Grey,9.42,바지 시력 관리 고글 색안경 검정색과 흰색 플래시 사진 스타일 안경 단색화 흑백 사진,드라이브 맛 트렌디 세련된 알앤비 국내 힙합,Unlock,NaN,차다빈,차다빈 - Unlock,R&B
4,Black,50.38,Grey,22.21,Grey,9.42,바지 시력 관리 고글 색안경 검정색과 흰색 플래시 사진 스타일 안경 단색화 흑백 사진,드라이브 맛 트렌디 세련된 알앤비 국내 힙합,Beautiful,NaN,basecamp,basecamp - Beautiful,R&B


In [69]:
# 필요한 열 제외
data = data.drop(columns=['curArtist', 'curTitle'])

# curExplain, curHead, curObject를 결합하여 새로운 컬럼 생성
data['combined'] = data['curExplain'].astype(str) + ' ' + data['curHead'].astype(str) + ' ' + data['curObject'].astype(str)

# 범주형 변수를 category 타입으로 변환
categorical_cols = ['curColor', 'curColor2', 'curColor3', 'genre', 'song']
for col in categorical_cols:
    data[col] = data[col].astype('category')

# 'song' 열을 Label Encoding
label_encoder = LabelEncoder()
data['song'] = label_encoder.fit_transform(data['song'])

# TF-IDF 벡터화
tfidf_vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2), min_df=2, max_df=0.8)
tfidf_matrix = tfidf_vectorizer.fit_transform(data['combined'])

# TF-IDF 결과를 데이터프레임으로 변환
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# 모델 학습에 사용할 데이터 준비 (curExplain, curHead, curObject, combined 열 제거)
X = pd.concat([data[['curColor', 'curPer', 'curColor2', 'curPer2', 'curColor3', 'curPer3', 'genre']].reset_index(drop=True), tfidf_df.reset_index(drop=True)], axis=1)
y = data['song']

# DMatrix 생성
dtrain = xgb.DMatrix(X, label=y, enable_categorical=True)

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

#### XGBoost 모델의 특징
- 효율적인 구현: XGBoost는 gradient boosting 알고리즘을 효율적으로 구현한 라이브러리입니다. 이를 통해 빠른 학습 속도와 높은 예측 성능을 달성할 수 있습니다.
- 다양한 작업 지원: XGBoost는 분류, 회귀, 순위 예측 등 다양한 기계학습 작업을 지원합니다.
- 자동 특성 중요도 계산: XGBoost는 자동으로 각 특성의 중요도를 계산하여 제공합니다. 이를 통해 특성 선택 및 모델 해석에 활용할 수 있습니다.
- 병렬 처리 지원: XGBoost는 병렬 처리를 지원하여 대용량 데이터에 대한 학습이 가능합니다.
#### XGBoost 모델의 하이퍼파라미터
- max_depth: 트리의 최대 깊이를 제한하는 하이퍼파라미터입니다. 과적합을 방지하기 위해 적절한 값을 선택해야 합니다.
- learning_rate: 각 트리에서 업데이트되는 가중치의 크기를 조절하는 하이퍼파라미터입니다. 작은 값을 사용하면 과적합을 방지할 수 있습니다.
- n_estimators: 생성할 트리의 개수를 지정하는 하이퍼파라미터입니다. 적절한 값을 선택하여 모델의 성능을 향상시킬 수 있습니다.
- min_child_weight: 리프 노드에서 필요한 최소 샘플 수를 지정하는 하이퍼파라미터입니다. 과적합을 방지하는 데 도움이 됩니다.

In [41]:
# 모델 파라미터 설정
params = {
    'objective': 'multi:softmax',
    'learning_rate': 0.05,
    'max_depth': 6,
    'num_class': len(label_encoder.classes_)
}

# 학습 시간 측정
start_time = time.time()
model = xgb.train(params, dtrain, num_boost_round=100)
end_time = time.time()

# 학습 시간
training_time = end_time - start_time
print(f'학습 시간: {training_time} 초')



# 모델 평가
y_pred = model.predict(dtest)
y_pred = y_pred.astype(int)

# 예측된 레이블을 원래 값으로 변환
y_pred_original = label_encoder.inverse_transform(y_pred)
y_test_original = label_encoder.inverse_transform(y_test)

# 평가 지표 출력
print("Classification Report:")
print(classification_report(y_test_original, y_pred_original))

print("Confusion Matrix:")
print(confusion_matrix(y_test_original, y_pred_original))

Fitting 3 folds for each of 9 candidates, totalling 27 fits


ValueError: n_splits=3 cannot be greater than the number of members in each class.

In [23]:
# 모델 저장
with open('xgboost_model4.pkl', 'wb') as f:
    pickle.dump(model, f)

In [ ]:
# 모델 평가
y_pred = model.predict(dtest)
y_pred = y_pred.astype(int)

# 예측된 레이블을 원래 값으로 변환
y_pred_original = label_encoder.inverse_transform(y_pred)
y_test_original = label_encoder.inverse_transform(y_test)

# 평가 지표 출력
print("Classification Report:")
print(classification_report(y_test_original, y_pred_original))

print("Confusion Matrix:")
print(confusion_matrix(y_test_original, y_pred_original))

In [ ]:


# 예측된 레이블을 원래 값으로 변환
y_pred_original = label_encoder.inverse_transform(y_pred.astype(int))

# 예측 결과 출력
print("예측된 결과 (원래 값으로 변환):")
print(y_pred_original)

In [ ]:
# TF-IDF와 레이블 인코더 저장
with open('tfidf_vectorizer4.pkl', 'wb') as f:
    pickle.dump(tfidf_vectorizer, f)

with open('tfidf_matrix4.pkl', 'wb') as f:
    pickle.dump(tfidf_matrix, f)

with open('label_encoder4.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

#### 입력값 예측 함수

In [24]:
import numpy as np

# 사용자 입력을 받아 예측하는 함수
def recommend_songs(model, label_encoder, tfidf_vectorizer, tfidf_matrix, input_keywords, input_genre, n_recommendations=15):
    # 입력 키워드를 TF-IDF 벡터화
    input_vector = tfidf_vectorizer.transform([input_keywords])
    
    # 코사인 유사도 계산
    cosine_sim = cosine_similarity(input_vector, tfidf_matrix)
    
    # 입력한 장르와 일치하는 인덱스 필터링
    genre_indices = data[data['genre'] == input_genre].index
    
    # 유사도가 높은 순서로 정렬된 인덱스
    sorted_indices = genre_indices[np.argsort(-cosine_sim[0][genre_indices])]
    
    # 상위 n개의 유사한 노래 추천
    recommended_song_indices = sorted_indices[:n_recommendations]
    recommended_songs = label_encoder.inverse_transform(data.loc[recommended_song_indices, 'song'])
    
    return recommended_songs

In [25]:
# 저장된 TF-IDF 벡터화 및 레이블 인코더 불러오기
with open('tfidf_vectorizer4.pkl', 'rb') as f:
    tfidf_vectorizer = pickle.load(f)

with open('label_encoder4.pkl', 'rb') as f:
    label_encoder = pickle.load(f)

with open('tfidf_matrix4.pkl', 'rb') as f:
    tfidf_matrix = pickle.load(f)

# 저장된 모델 불러오기
with open('xgboost_model4.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

In [31]:
# 사용자 입력 예제
input_keywords = '신나는 운동중 바다'
input_genre = '발라드'

# 예측 수행
recommended_songs = recommend_songs(loaded_model, label_encoder, tfidf_vectorizer, tfidf_matrix, input_keywords, input_genre, n_recommendations=15)
print("추천된 노래들:")
for song in recommended_songs:
    print(song)

추천된 노래들:
정아로 - 아닌데
선예 - 울지 않을게
비비 - 우리가 헤어져야 했던 이유
밀리그램 - 애벌레
카더가든 - 그대 작은 나의 세상이 되어
죠지 - 바라봐줘요
SOLE - 아무말도
BehindtheMoon - 너만의나
라디 - 들어봐줘요
조소정 - 물음표가 가득한 나의 밤
CHEEZE - 퇴근시간
nov - 수고했어요
선예 - 울지 않을게
DOKO - 날 사랑하지 말아요
임한별 - 안녕, 오늘의 그대에게


In [32]:
# 추천된 결과를 평가하는 함수
def evaluate_recommendations(recommended_songs, actual_songs):
    intersection = set(recommended_songs).intersection(set(actual_songs))
    accuracy = len(intersection) / len(actual_songs)
    precision = len(intersection) / len(recommended_songs)
    recall = len(intersection) / len(actual_songs)
    if precision + recall == 0:
        f1 = 0
    else:
        f1 = 2 * (precision * recall) / (precision + recall)
    return accuracy, precision, recall, f1

# 실제 장르 내 노래 목록
actual_songs = label_encoder.inverse_transform(data[data['genre'] == input_genre]['song'])

# 평가 수행
accuracy, precision, recall, f1 = evaluate_recommendations(recommended_songs, actual_songs)
print(f'추천 시스템 정확도: {accuracy}')
print(f'추천 시스템 정밀도: {precision}')
print(f'추천 시스템 재현율: {recall}')
print(f'추천 시스템 F1 점수: {f1}')

추천 시스템 정확도: 0.023064250411861616
추천 시스템 정밀도: 0.9333333333333333
추천 시스템 재현율: 0.023064250411861616
추천 시스템 F1 점수: 0.04501607717041801
